In [37]:
dayObs = '20241028'
run = "None"
sensor='R10_S10'

In [38]:
from lsst.daf.butler import Butler
from astropy.table import Table
from astropy.time import Time, TimeDelta
import tabulate
import numpy

In [39]:
if dayObs=='Today':
    dayObs = (Time.now() - TimeDelta(0.5, format='jd')).iso[:10].replace('-','')

In [40]:
butler = Butler('embargo_new', collections=[
    'LSSTCam/raw/all',
    'LSSTCam/calib/unbounded',
    'LSSTCam/photodiode'
])

In [41]:
exposureList = []

if run == "None":
    where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"
else:
    where=f"exposure.science_program='{run}' and instrument='LSSTCam'"    

for record in butler.registry.queryDimensionRecords("exposure", where=where):
    nimage=len(list(butler.registry.queryDatasets('raw',dataId=record.dataId)))
    hasPD=len(list(butler.registry.queryDatasets('photodiode',dataId=record.dataId)))
    exposureList.append([record.id, record, nimage, hasPD])
exposureList.sort(key=lambda x: x[0])
table = Table(rows=[ (
            f'<a href=http://lsstcam-mcm.cp.lsst.org/FITSInfo/view.html?image={record.obs_id} target="_blank">{record.obs_id}</a>',
            record.timespan.begin.iso,
            record.seq_num,
            record.observation_type,
            record.science_program,
            record.observation_reason,
            record.exposure_time,
            record.physical_filter,
            nimage,
            f'<a href=https://usdf-rsp-dev.slac.stanford.edu/times-square/github/lsst/CameraTimesSquare/Run7/CheckPDTraces?exposure={record.id} target="_blank" >PD</a>' if hasPD else "",
            f'<a href=https://usdf-rsp-dev.slac.stanford.edu/times-square/github/lsst/CameraTimesSquare/Run7/FitsHeader?expid={record.obs_id}&detector={sensor} target="_blank" >meta</a>',    
                     )
            for [id,record,nimage, hasPD] in exposureList ]
            )

In [42]:
subtable=Table(rows=[  ( record.obs_id, record.seq_num, nimage ) for [id,record,nimage, hasPD] in exposureList ], names=(  'id', 'seqnum', 'nimage'  ) )

# Possible data transfer issue

In [43]:
for a, b in zip(subtable[:-1][numpy.diff(subtable['seqnum'])!=1], subtable[1:][numpy.diff(subtable['seqnum'])!=1]):
    print(a['id'], b['id'])

# Changes in number of images ingested

In [44]:
for a, b in zip(subtable[:-1][numpy.diff(subtable['nimage'])!=0], subtable[1:][numpy.diff(subtable['nimage'])!=0]):
    print(a['id'], a['nimage'], b['id'], b['nimage'])

# Full records

In [45]:
print(f"The number of exposures is {len(table)}")
tabulate.tabulate(table[::-1], tablefmt='unsafehtml',headers=[
                   'obs_id', 'time', 'seqnum', 'observation_type',
                   'science_program', 'observation_reason','exposure_time',
                   'physical_filter', 'nimage', 'hasPD', 'metadata'
                  ])

The number of exposures is 500


obs_id,time,seqnum,observation_type,science_program,observation_reason,exposure_time,physical_filter,nimage,hasPD,metadata
MC_C_20241028_000500,2024-10-28 17:54:19.754008,500,dark,E1812,bot_persistence,15,unknown,197,,meta
MC_C_20241028_000499,2024-10-28 17:54:02.057987,499,dark,E1812,bot_persistence,15,unknown,197,,meta
MC_C_20241028_000498,2024-10-28 17:53:44.310990,498,dark,E1812,bot_persistence,15,unknown,197,,meta
MC_C_20241028_000497,2024-10-28 17:53:26.296000,497,dark,E1812,bot_persistence,15,unknown,197,,meta
MC_C_20241028_000496,2024-10-28 17:53:08.487004,496,dark,E1812,bot_persistence,15,unknown,197,,meta
MC_C_20241028_000495,2024-10-28 17:52:50.520012,495,dark,E1812,bot_persistence,15,unknown,197,,meta
MC_C_20241028_000494,2024-10-28 17:52:32.615985,494,dark,E1812,bot_persistence,15,unknown,197,,meta
MC_C_20241028_000493,2024-10-28 17:52:14.873012,493,dark,E1812,bot_persistence,15,unknown,197,,meta
MC_C_20241028_000492,2024-10-28 17:51:57.062004,492,dark,E1812,bot_persistence,15,unknown,197,,meta
MC_C_20241028_000491,2024-10-28 17:51:39.482980,491,dark,E1812,bot_persistence,15,unknown,197,,meta
